# IN_SYS - SW14 Exercise 2

## Functions calling

In [1]:
import os
import json
import requests
import openai

from dotenv import load_dotenv
from docx import Document
import os

# Load OpenAI API key
load_dotenv()

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
fx_key = os.getenv("FX_API_KEY")

In [ ]:
def get_exchange_rates(from_currency: str, to_currency: str) -> dict:
    """
    Calls the ExchangeRate-API to get the latest exchange rates for given symbols.
    symbols should be a comma-separated string of currency codes, e.g. "from USD to CHF".
    """
    url = f"https://v6.exchangerate-api.com/v6/{fx_key}/pair/{from_currency}/{to_currency}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()  # returns {...'time_last_update_utc': 'Thu, 18 Dec 20250', 'base_code': 'USD', 'target_code': 'CHF', 'conversion_rate': 0.7311, ...}
    else:
        return {"error": "Unable to fetch exchange rates"}

# Define the function schema for OpenAI. This schema is used by the model to generate structured JSON
functions = [
    {
        "name": "get_exchange_rates",
        "description": "Fetch the latest exchange rate between two specific currencies.",
        "parameters": {
            "type": "object",
            "properties": {     # this defines each parameter
                "from_currency": {
                    "type": "string",
                    "description": "The source currency symbol (3-letter code). Example: 'USD', 'EUR'"
                },
                "to_currency": {
                    "type": "string",
                    "description": "The target currency symbol (3-letter code). Example: 'JPY', 'GBP'"
                }
            },
            "required": ["from_currency", "to_currency"]
        }
    }
]

while True:
    user_message = input("Please enter two currencies for which you would like to get the exchange rate in three-letters format (or 'exit' to quit): ")
    if user_message.strip().lower() == "exit":
        print("Exiting the loop.")
        break
    
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": user_message}],
        functions=functions,
        function_call="auto"
    )

    msg = response.choices[0].message

    if msg.function_call is not None:
        # The model wants to call the function
        function_name = msg.function_call.name
        function_args = json.loads(msg.function_call.arguments)
        exchange_data = get_exchange_rates(**function_args)
        
        if function_name == "get_exchange_rates":
            # Call the Python function
            exchange_data = get_exchange_rates(**function_args)

            # Send the function's result back to the model
            follow_up_response = openai.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "user", "content": user_message},
                    msg.to_dict(),
                    {
                        "role": "function",
                        "name": function_name,
                        "content": json.dumps(exchange_data)
                    }
                ]
            )

            # Print the final answer from the assistant
            print("\n\nAssistant:", follow_up_response.choices[0].message.content, "\n\n")
    else:
        # The model didn't call a function and responded directly
        print("\n\nAssistant:", msg.content, "\n\n")


Please enter two currencies for which you would like to get the exchange rate in three-letters format (or 'exit' to quit):  from CHF to USD




Assistant: The current exchange rate from Swiss Franc (CHF) to US Dollar (USD) is 1.2568 USD for each CHF. Please note that exchange rates are subject to change. 


